# Preparing Libraries and Installations


In [1]:
%pip install bs4 selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/484.6 kB ? eta -:--:--
   ---------------------- ----------------- 276.5/484.6 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 484.6/484.6 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ---------------------------------------- 307.7/307.7 kB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import (
    StaleElementReferenceException,
)

from bs4 import BeautifulSoup

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

import pandas as pd
import time

# Scraping the Website

In [2]:
# URL Ulasan dari Female Daily
base_url = "https://reviews.femaledaily.com/products/moisturizer/sun-protection-1/emina/sun-protection-spf-30-pa?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=oldest&page={}"
page = 412
reviews_count = 1

# Koneksi ke mongodb
username = "ayukrisna"
password = "bigdata"
host = "ayukrisna.c81wxan.mongodb.net"
dbname = "femaledaily_review"
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

try:
    client = MongoClient(uri)
    db = client[dbname]
    collection = db['raw_data_duplicated']

    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}")
    exit()

try:
    # Inisialisasi option chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Inisialisasi WebDriver
    service = Service(executable_path = ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    #data frame
    df_data = pd.DataFrame(columns = ['date', 'username', 'age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point', 'likes', 'comments'])

    # Scraping data
    while (page < 500):
        # Buka URL
        url = base_url.format(page)
        driver.get(url)
        print(page)
        wait = WebDriverWait(driver, 10)

        # Ekstrak dan print title
        # print(driver.title)
        time.sleep(5) #tunggu 5 detik

        # Get List Reviews
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        list_reviews = soup.find(class_='list-reviews')

        if not list_reviews:
            print("No reviews found on this page.")
            page += 1
            continue

        reviews = list_reviews.find_all(class_='review-card')
                       
        for review in reviews:
            #date
            date_element = review.find(class_='review-date')
            date = date_element.get_text(strip=True) if date_element else ""

            #left column
            left_col = review.find(class_='review-card-col left')
            username_element = left_col.find('p', class_='profile-username')
            age_element = left_col.find('p', class_='profile-age')
            skin_type_element = left_col.find('p', class_='profile-description')

            # Extract username, age, and skin type only if elements are found
            username = username_element.get_text(strip=True) if username_element else ""
            age = age_element.get_text(strip=True) if age_element else ""
            skin_type = skin_type_element.get_text(strip=True) if skin_type_element else ""

            #right column
            right_col = review.find(class_='review-card-col right')
            wrapper = right_col.find(class_='review-content-wrapper')

            #start ratings
            star_icons = wrapper.find_all('i', class_='icon-ic_big_star_full')
            rating = len(star_icons)

            #recommend
            recommendation_tag = wrapper.find('p', class_='recommend')
            if recommendation_tag:
                if 'recommends' in recommendation_tag.get_text():
                    recommended = "recommended"
                else:
                    recommended = "not recommended"
            else:
                recommended = ""

            #text review
            try:
                review_text_element = wrapper.find(class_='text-content')
                review_text = review_text_element.get_text(strip=True) if review_text_element else ""
            except (StaleElementReferenceException, NoSuchElementException):
                review_text = ""

            # Usage period and purchase point
            info_wrapper = wrapper.find(class_='information-wrapper')
            usage_period = ""
            purchase_point = ""
            if info_wrapper:
                usage_period_element = info_wrapper.find('span', string='Usage Period ')
                purchase_point_element = info_wrapper.find('span', string='Purchase Point ')

                # Extract usage period and purchase point only if elements are found
                if usage_period_element:
                    usage_period = usage_period_element.find_next('b').get_text(strip=True)
                if purchase_point_element:
                    purchase_point = purchase_point_element.find_next('b').get_text(strip=True)
            
            # Likes and dislikes
            comment_section = wrapper.find(class_='comment-section')
            likes = ""
            comments = ""
            if comment_section:
                likes_element = comment_section.find(class_= "like icon")
                comments_element = comment_section.find(class_= "chat icon")

                # Extract likes and comments only if elements are found
                if likes_element:
                    likes = likes_element.get_text(strip=True)
                if comments_element:
                    comments = comments_element.get_text(strip=True)

            # simpan ke frame
            df_data.loc[len(df_data)] = [date, username, age, skin_type, rating, recommended, review_text, usage_period, purchase_point, likes, comments]

            #simpan ke mongodb
            review_data = {
                'date': date,
                'username': username,
                'age': age,
                'skin_type': skin_type,
                'rating': rating,
                'recommended': recommended,
                'review': review_text,
                'usage_period': usage_period,
                'purchase_point': purchase_point,
                'likes': likes,
                'comments': comments
            }
            collection.insert_one(review_data)
            print(f"Successfully inserted {reviews_count}-reviews")
            reviews_count += 1

        # Memeriksa apakah ada button next page
        try:
            next_button = driver.find_element(By.ID, 'id_next_page')
            if 'paging-prev-text-inactive' in next_button.get_attribute('class'):
                break
        except NoSuchElementException:
            break

        page += 1
        time.sleep(2)

#exception
except WebDriverException as e:
    print(f"WebDriverException: {e}")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")
# except TimeoutException:
#     print("The element was not clickable within the given time")
finally:
    driver.quit()

pd.set_option('display.max_colwidth', None)
df_data

Successfully connected to MongoDB!
412
Successfully inserted 1-reviews
Successfully inserted 2-reviews
Successfully inserted 3-reviews
Successfully inserted 4-reviews
Successfully inserted 5-reviews
Successfully inserted 6-reviews
Successfully inserted 7-reviews
Successfully inserted 8-reviews
Successfully inserted 9-reviews
Successfully inserted 10-reviews
413
Successfully inserted 11-reviews
Successfully inserted 12-reviews
Successfully inserted 13-reviews
Successfully inserted 14-reviews
Successfully inserted 15-reviews
Successfully inserted 16-reviews
Successfully inserted 17-reviews
Successfully inserted 18-reviews
Successfully inserted 19-reviews
Successfully inserted 20-reviews
414
Successfully inserted 21-reviews
Successfully inserted 22-reviews
Successfully inserted 23-reviews
Successfully inserted 24-reviews
Successfully inserted 25-reviews
Successfully inserted 26-reviews
Successfully inserted 27-reviews
Successfully inserted 28-reviews
Successfully inserted 29-reviews
Succe

,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,17 May 2019,misslarasati,30 - 34,"Combination, Medium Light, Warm",4,,"biasa aja sih ini di gw, ga bikin breakout, cuma agak lengket aja menurut gw yg kulitnya oily, agak lama meresapnya.terus banyak isinya dgn hrg yg sangat affordable.bakal repurchase? maybe yes maybe no.",,,2,0
1,17 May 2019,nadhiroh,19 - 24,"Oily, Medium Dark, Warm",4,,"sunsceen ini teksturnya kental agak cair, untuk tipe kulit berminyak cocok aja menurutku, tapi kecepatan untuk menyerap dikulit butuh beberapa waktu dan perlu di diamkan. untuk pemakaiannya pastikan dulu wajah harus bersihbau dari produk ini enak sih menurutku dan jadi seger gitu kalo abis di aplikasikan",,,0,0
2,17 May 2019,luthfiahrnr_,19 - 24,"Combination, Medium, Neutral",5,,"beli sunscreen ini karna harganya affordable dan identik sama remaja. udah pake ini di botol ketiga saking sukanya karena gak bikin breakout, ga lengket, di muka juga ringan, dan ga bikin minyakan parah kalo uda beberapa jam",,,0,0
3,17 May 2019,fardew,19 - 24,"Combination, Medium, Warm",5,,"udah repurchase berkali kali, karena harganya yg pas di kantong ya kan drpd SB merk lainnyaa. ringan juga dipake, cepet ngeresap, packagingnya juga kyutt. Yang penting ga bikin wajah BO / jd kilang minyak",,,0,0
4,17 May 2019,septiliariscanova,19 - 24,"Normal, Medium, Neutral",5,,"Salah satu product sun protection yang affordable yang udah 3 kali di repurchase, gak ninggalin whitecast di muka sukakkk pokoknya, ampe udah ngeracunin ke sodara-sodara ku yang lain juga, ini andalan banget kalo kemana-mana kelupaan bisa stress wkwkwk",,,0,0
...,...,...,...,...,...,...,...,...,...,...,...
874,03 Jul 2019,helvyradhina,19 - 24,"Oily, Medium Light, Warm",3,,gasuka baunya kaya jeruk kematengan :( dan pas dipake ke muka tuh kaya berminya banget jd ga nyaman gitu. jd kupake buat tangan sama kaki aja klo lg kegiatan outdoor. lumayan buat menghindari belang hehe,,,1,0
875,03 Jul 2019,febella,19 - 24,"Oily, Light, Warm",5,,"Wow banget sih dengan harga segini! Ini tuh murah, tapi kualitas nggak murahan. Waktu awal-awal di-apply emang agak lama nyerepnya, tapi its okay untuk harga segini. Minus-nya cuma di baunya aja sih. Selebihnya, bagus banget nget ngettt! Sejak pakai ini, mukaku jadi nggak pernah terbakar lagi.",,,0,2
876,04 Jul 2019,divaputri96,19 - 24,"Oily, Medium Light, Cool",4,,"ini sunscreen pertama yg ak cb,meskioun bnyk jg yg blng bkl bkin kusam tp untng di ak gk,pas dipake tu enak ada snsasi dingin ,cepet nyerap,dn gk lengket.suka lah pkoknya.semakin hari semakin di pake juga ak ngeliat ad perubahan sma kulitku termasuk kalo siang bolong habis anter anak muka gak gitu gosong sih.klo pas dirumah aja ak tetep pake karena kan infonya panas dari alat masak juga pengaruh ke muka ya.mknya ak pake tiap hari pagi sore. hehe",,,0,0
877,04 Jul 2019,Febbytr,19 - 24,"Combination, Medium Light, Warm",5,,"sebagus itu sih, sunscreennya. gak lengket gitu.. tapi aku kurang suka wangi - wanginya gitu. tapi gak masalah sih yang penting ini bagus kalo kita mau berpergian jika cuacanya sedang panas. kurangnyaa, tubenya terlalu besar jadi suka susah buat dibawa di tas gitu kalo mau pergi pergi. dan ini awet gitu sih bagus lah pokoknya",,,0,0


# Delete Duplicate

In [3]:
all_data = list(collection.find())
df_data = pd.DataFrame(all_data)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,664ac96a7506cd60f11a6df3,14 hours ago,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,"Waktu smp aku pakai ss ini selama ±2 tahun, terus aku stop selama covid karena ga pernah keluar. Pas coba pake lagi, tiba2 breakout parah banget sehari setelahnya, padahal exp product masih lama. Dari aman selama dua tahun, terus tiba-tiba ga cocok dan breakout parah yg jerawatan batu gitu. I don't know why, sepertinya kulitku jadi sensitif dengan fragrancenya. Tapi untuk yg kulit normal, ini oke oke aja kayanya.",More than 1 year,Shopee,0,0
1,664ac96c7506cd60f11a6df4,2 days ago,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,"Aku ga terlalu cocok pake produk ini terlebih lagi muka aku oily skin jadi dipake 2 kali langsungkeluar jerawatnya, mungkin untuk kulit yang dryskin bisa di coba tapi aku cukup sekali pake produk ini karena kalo ga cocok bikin kulit jerawat besar²",Less than 1 week,Shopee,1,0
2,664ac96c7506cd60f11a6df5,2 days ago,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, first apply aku kurang suka sama fragrance nya sangat kencang, dia teksturnya gel gitu dan gampang di apply di kulit. Dan kalian tau aku udah 4x repurchase karna harganya yang affordable sewaktu aku masih sekolah. Untuk hasil dari pemakaian ntah kenapa aku kyk ngerasa keringettan gitu ya huhu",3 months - 6 months,Shopee,0,0
3,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,"Teksturnya agak watery gituDiblendnya agak lumayan usahaDi muka saya, kerasa panas pas dipakai. Dan ada aroma agak tajemnya. Tapi emang harganya sangat affordable. Kalau pas keringetan, lumayan keliatan putih putih gitu bulir keringetnya dan agak pedih dimata.",1 week - 1 month,Super Indo,0,0
4,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,"udah pake ini dari jaman SMA hahah seneng banget bisa cocok. suka banget sama texturenya yang ringan, gampang nyerep, gampang diblend, gak bikin gerah, gak ada whitecast. yang bikin seneng lagi, harganya murah meriah dan gampang didapetin.",More than 1 year,Alfamart,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14941,664b1e0e5e454cecdf3f5dda,03 Jul 2019,helvyradhina,19 - 24,"Oily, Medium Light, Warm",3,,gasuka baunya kaya jeruk kematengan :( dan pas dipake ke muka tuh kaya berminya banget jd ga nyaman gitu. jd kupake buat tangan sama kaki aja klo lg kegiatan outdoor. lumayan buat menghindari belang hehe,,,1,0
14942,664b1e0e5e454cecdf3f5ddb,03 Jul 2019,febella,19 - 24,"Oily, Light, Warm",5,,"Wow banget sih dengan harga segini! Ini tuh murah, tapi kualitas nggak murahan. Waktu awal-awal di-apply emang agak lama nyerepnya, tapi its okay untuk harga segini. Minus-nya cuma di baunya aja sih. Selebihnya, bagus banget nget ngettt! Sejak pakai ini, mukaku jadi nggak pernah terbakar lagi.",,,0,2
14943,664b1e0e5e454cecdf3f5ddc,04 Jul 2019,divaputri96,19 - 24,"Oily, Medium Light, Cool",4,,"ini sunscreen pertama yg ak cb,meskioun bnyk jg yg blng bkl bkin kusam tp untng di ak gk,pas dipake tu enak ada snsasi dingin ,cepet nyerap,dn gk lengket.suka lah pkoknya.semakin hari semakin di pake juga ak ngeliat ad perubahan sma kulitku termasuk kalo siang bolong habis anter anak muka gak gitu gosong sih.klo pas dirumah aja ak tetep pake karena kan infonya panas dari alat masak juga pengaruh ke muka ya.mknya ak pake tiap hari pagi sore. hehe",,,0,0
14944,664b1e0e5e454cecdf3f5ddd,04 Jul 2019,Febbytr,19 - 24,"Combination, Medium Light, Warm",5,,"sebagus itu sih, sunscreennya. gak lengket gitu.. tapi aku kurang suka wangi - wanginya gitu. tapi gak masalah sih yang penting ini bagus kalo kita mau berpergian jika cuacanya sedang panas. kurangnyaa, tubenya terlalu besar jadi suka susah buat dibawa di tas gitu kalo mau pergi pergi. dan ini awet gitu sih bagus la

In [4]:
df_data.drop_duplicates(subset=['date', 'username', 'age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point', 'likes', 'comments'], keep='first', inplace=True)
df_data

,_id,date,username,age,skin_type,rating,recommended,review,usage_period,purchase_point,likes,comments
0,664ac96a7506cd60f11a6df3,14 hours ago,mtraandita,19 - 24,"Combination, Medium, Warm",3,not recommended,"Waktu smp aku pakai ss ini selama ±2 tahun, terus aku stop selama covid karena ga pernah keluar. Pas coba pake lagi, tiba2 breakout parah banget sehari setelahnya, padahal exp product masih lama. Dari aman selama dua tahun, terus tiba-tiba ga cocok dan breakout parah yg jerawatan batu gitu. I don't know why, sepertinya kulitku jadi sensitif dengan fragrancenya. Tapi untuk yg kulit normal, ini oke oke aja kayanya.",More than 1 year,Shopee,0,0
1,664ac96c7506cd60f11a6df4,2 days ago,Miftahuljahra,19 - 24,"Combination, Medium Light, Cool",2,not recommended,"Aku ga terlalu cocok pake produk ini terlebih lagi muka aku oily skin jadi dipake 2 kali langsungkeluar jerawatnya, mungkin untuk kulit yang dryskin bisa di coba tapi aku cukup sekali pake produk ini karena kalo ga cocok bikin kulit jerawat besar²",Less than 1 week,Shopee,1,0
2,664ac96c7506cd60f11a6df5,2 days ago,Marizka_nuril,19 - 24,"Combination, Medium, Neutral",3,not recommended,"Sunscreen pertama aku sewaktu kenal skincare, first apply aku kurang suka sama fragrance nya sangat kencang, dia teksturnya gel gitu dan gampang di apply di kulit. Dan kalian tau aku udah 4x repurchase karna harganya yang affordable sewaktu aku masih sekolah. Untuk hasil dari pemakaian ntah kenapa aku kyk ngerasa keringettan gitu ya huhu",3 months - 6 months,Shopee,0,0
3,664ac96c7506cd60f11a6df6,3 days ago,Denyul,30 - 34,"Combination, Medium Light, Cool",3,recommended,"Teksturnya agak watery gituDiblendnya agak lumayan usahaDi muka saya, kerasa panas pas dipakai. Dan ada aroma agak tajemnya. Tapi emang harganya sangat affordable. Kalau pas keringetan, lumayan keliatan putih putih gitu bulir keringetnya dan agak pedih dimata.",1 week - 1 month,Super Indo,0,0
4,664ac96d7506cd60f11a6df7,3 days ago,stephaniekumala,19 - 24,"Oily, Medium, Neutral",5,recommended,"udah pake ini dari jaman SMA hahah seneng banget bisa cocok. suka banget sama texturenya yang ringan, gampang nyerep, gampang diblend, gak bikin gerah, gak ada whitecast. yang bikin seneng lagi, harganya murah meriah dan gampang didapetin.",More than 1 year,Alfamart,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14331,664b1b6b5e454cecdf3f5b78,31 May 2019,anitaintan,25 - 29,"Combination, Medium, Warm",2,,"Produk SS pertama yg aku coba.Beli ini karena baca review katanya bagus.tapi, kalau dipake bareng BB cream, aduh wajah jadi aneh banget, jadi abu-abu. Kalau pake ini, dari pelembab, langsung SS. plus di wajahku jadi berminyak, walaupun bisa di set pake bedak tabur.Wanginya aku kurang suka. Aneh menurutku.kalau dipake sesuai rekomendasi (2 ruas jari), berat banget rasanya wajah, apalagi keringetan, kelihatan banget putih-nya.Repurchase? No, mau berpetualang nyari SS lain, tapi karena nggk bikin bruntusan apa jerawat, dipake smp habis dulu.",,,0,0
14332,664b1b6c5e454cecdf3f5b79,31 May 2019,HanPHLP,25 - 29,"Combination, Medium Light, Warm",2,,"sebenernya nyoba beli sunscreen ini karena penasaran. tapi sayang, pas dicoba di kulitku gak cocok. jd berminyak dan bikin bruntusan. wanginya juga klo menurutku agak aneh. keganggu aja gtu sama wanginya. jd lgsg ku kasih ke adikku. di kulit dia sih cocok yg type normal skin.",,,0,0
14333,664b1b6c5e454cecdf3f5b7a,31 May 2019,Putrialyam,19 - 24,"Combination, Medium, Warm",3,,"Beli karena pas promo dan kebetulan sunscreen yg biasa dipake lg habis. Kemasan lucu dan kecil jd praktis dibawa kemana2. Sebenarnya suka sama sunscreen ini karena dia diblende nya gampang, wangi nya enak kyk kelapa. Cmn lama2 aku pake jd kusam di muka aku-_-",,,0,0
14404,664b1bbb5e454cecdf3f5bc1,05 Jun 2019,jovriskae,18 and Under,"Combination, Medium, Cool",2,,"ini sunscreen pertamaku, awalnya pake ini karna kata temen kulit bener bener butuh bgt sunscreen jadi kubeli ini yg samaan sama temen ceritanya karna aku masi ga ngerti

In [5]:
collection = db['raw_data']
collection.insert_many(df_data.to_dict('records'))

InsertManyResult([ObjectId('664ac96a7506cd60f11a6df3'), ObjectId('664ac96c7506cd60f11a6df4'), ObjectId('664ac96c7506cd60f11a6df5'), ObjectId('664ac96c7506cd60f11a6df6'), ObjectId('664ac96d7506cd60f11a6df7'), ObjectId('664ac96d7506cd60f11a6df8'), ObjectId('664ac96d7506cd60f11a6df9'), ObjectId('664ac96d7506cd60f11a6dfa'), ObjectId('664ac96d7506cd60f11a6dfb'), ObjectId('664ac96d7506cd60f11a6dfc'), ObjectId('664ac97e7506cd60f11a6dfd'), ObjectId('664ac97e7506cd60f11a6dfe'), ObjectId('664ac97e7506cd60f11a6dff'), ObjectId('664ac97e7506cd60f11a6e00'), ObjectId('664ac97e7506cd60f11a6e01'), ObjectId('664ac97e7506cd60f11a6e02'), ObjectId('664ac97e7506cd60f11a6e03'), ObjectId('664ac97e7506cd60f11a6e04'), ObjectId('664ac97e7506cd60f11a6e05'), ObjectId('664ac97e7506cd60f11a6e06'), ObjectId('664ac98a7506cd60f11a6e07'), ObjectId('664ac98a7506cd60f11a6e08'), ObjectId('664ac98a7506cd60f11a6e09'), ObjectId('664ac98a7506cd60f11a6e0a'), ObjectId('664ac98a7506cd60f11a6e0b'), ObjectId('664ac98a7506cd60f11a6e